# Clustering

<img src="./img/6_classification_vs_clustering.png" width="700px"><br><br>

With `Classification` we want to find a hyperplane that __best separates__ data into similar categories.  
Classification requires __labeled__ data.

With `Clustering` we want to group __unlabeled__ data into __similar__ sets of examples.  
Before you can classify or group similar examples, you need to define __similarity__.

<br>

### Motivation

Assume a set of unlabeled data. You may want to find similarities (within the features).

<img src="./img/6_Clustering-example-with-intra-and-inter-clustering-illustrations.png" width="500px"><br>

<span style="font-size: 70%">Source: <a href="https://www.researchgate.net/figure/Clustering-example-with-intra-and-inter-clustering-illustrations_fig1_344590665">ResearchGate</a></span><br>

Examples:

- market segmentation (buyers segmented by income)
- social network analysis (sentiment analysis of contributions)
- recommendation groupings (commerce applications)
- astronomy (dark matter detection, categorisation of stars by brightness and red-shift)
- ...


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load sample data-set
X = np.loadtxt('./data/data_clustering.txt', delimiter=',')

# Plot input data
plt.scatter(X[:, 0], X[:, 1])
plt.show()

Our goal is to identify the intrinsic properties of data points that make them belong to the same group - `clusters`. 

In order to find these `clusters`, we use some kind of __similarity measures__ such as __Euclidean distance__. These similarity measure can estimate the tightness of a cluster.  

There is no universal similarity metric that works for all the cases.

It depends on the problem at hand. For example, we might be interested in finding the representative data point for each group or we might be interested in finding the outliers in our data. Depending on the situation, we will end up choosing the appropriate metric.

# k-Means Cluster

A simple but effective clustering algorithm is `k-Means`.

`k-means` follows these steps:

> &nbsp;
> 1. define $ k  \gt 1 $ to group data into $ k $ clusters ($ k $ ... hyperparameter)<br>  
> 2. define $ k $ _cluster centroids_ $ c_i $ (either manually or algorithmically) ... $ max \left( distance \left( k_i - k_j \right) \right) $  
_repeat until equilibrium is reached:_  
> 3. &nbsp;&nbsp;&nbsp;&nbsp;calculate minimum distance of data points to clusters and assign each point a _centroid_ based on the distance $ c_x = min_{i=1..k} \left( x_j - c_i \right) $
> 4. &nbsp;&nbsp;&nbsp;&nbsp;calculate the mean of all data points in a cluster and assign this mean as the new centroid  
> <br><br>

These $ k $ _centroids_ are used for inference (prediction).
<br><br>

<img src="./img/6_K-means_convergence.gif"><br><br>
<span style="font-size: 70%">k-Means convergence over the iterations. If the centroids don't change beyond a threshold (<b>iterations 12-14</b>), the algorithm has converged and stops.<br>
Source: <a href="https://en.wikipedia.org/wiki/K-means_clustering">Wikipedia</a></span>
<br><br>

Let's try `k-Means`.

In [ ]:
# the obligatory imports
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans

from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import silhouette_score

In [ ]:
# Load data
X = np.loadtxt('./data/data_clustering.txt', delimiter=',')

# prepare data
X_train, X_test = train_test_split(X, test_size=0.2, random_state=1)

In [ ]:
# define k as hyperparameter, take a wild guess
k = 5

# model generation
kM = KMeans(n_clusters=k, n_init="auto")
kM.fit(X_train)
train_labels = kM.labels_

In [ ]:
# evaluate model
test_labels = kM.predict(X_test)

In [ ]:
# visualizing data and decision boundaries
DecisionBoundaryDisplay.from_estimator(kM, X, alpha=0.4, response_method="predict")
plt.scatter(X_train[:, 0], X_train[:, 1], c=train_labels)
plt.scatter(X_test[:, 0], X_test[:, 1], c=test_labels, alpha=0.5, edgecolors="#666666")
plt.scatter(kM.cluster_centers_[:, 0], kM.cluster_centers_[:, 1], c="#ff0000", marker="x", s=140)
plt.title("")
plt.show()

### Finding the best $ k $

What is the best $ k $ ?

If the data is naturally organized into a number of distinct clusters, then it is easy to visually examine it and draw some inferences. But this is rarely the case in the real world. The data in the real world is huge and messy. So we need a way to quantify the quality of the clustering.

##### Silhouette score

`Silhouette` refers to a method used to check the consistency of clusters in our data. It gives an estimate of how well each data point fits with its cluster.

The `silhouette score` is a __metric__ that measures how similar a data point is to its own cluster, as compared to other clusters. The silhouette score works with any similarity metric.
<br><br>

For each data point, the _silhouette_score_ is computed:

> &nbsp;  
> $ silhouette\_score = \left( p - q \right) / max \left( p, q \right) $
> <br><br>

with:

$ p $ ... mean distance to the points in the nearest cluster that the data point is not a part of  
$ q $ ... mean intra-cluster distance to all the points in its own cluster.

The value of the _silhouette_score_ lies between [-1, 1]. 
<br><br>


A score closer to 1 indicates that the data point is very similar to other data points in the cluster, whereas a score closer to -1 indicates that the data point is not similar to the data points in its cluster. 

One way to think about it is if you get too many points with negative silhouette scores, then we may have too few or too many clusters in our data. We need to run the clustering algorithm again to find the optimal number of clusters.

Let's see how to estimate the clustering performance using silhouette scores.

In [ ]:
# we use the data above (X, X_train, X_test)
values = np.arange(2, 16)
scores = []

# Iterate through the defined range
for k in values:
    kM = KMeans(n_clusters=k, n_init="auto")
    kM.fit(X_train)
    score = silhouette_score(X_train, kM.labels_, metric='euclidean', sample_size=len(X_train))
    scores.append([k, score])

# find best k and plot
ss = np.array(scores)
max_ss = max(ss[:, 1])
best = [i for i, ss in enumerate(ss[:, 1]) if ss == max_ss][0]
best_k, best_ss = ss[best, 0], ss[best, 1]
plt.plot(ss[:, 1], ss[:, 0])
plt.scatter(best_ss, best_k, c="white", s=80, edgecolors="red")
plt.title(f"Best silhouette score: {best_ss:.3f} for k={best_k:.0f}")
plt.show()

This verifies our initial guess, that our data contains ___5___ clusters.

#### Inner inertia

Assuming the data points of a cluster should be "close" to the cluster centers (and the clusters should be "far" from each other, i.e. well-separated).

This means that the __inner variance__ (= sum of squared distances of data points to cluster center) should be as small as possible. 

The total variance is returned as `inertia`.

In [ ]:
# calculate the total variance and visualize inertia
total_variance = [KMeans(n_clusters=k, n_init="auto").fit(X_train).inertia_ for k in values]

plt.plot(total_variance)
plt.title(f"The gradient of total variance changes most at k={best_k:.0f}")
plt.scatter(3, total_variance[3], c="white", s=80, edgecolors="red")
plt.show()

The optimum number of clusters can be found where the inertia changes its gradient significantly (where the graph "bends").

Computing the optimum number of clusters from the inertia requires some Python programming (that we leave to the student as an exercise).

<table>
<tr>
<td style="border-style: none"><img src="./img/0_students_input.png" height="100px"></td>
<td style="border-style: none">&nbsp;&nbsp;</td>
<td style="border-style: none; vertical-align: middle"><h5>Students task:</h5>Program:
<ul>
    <li>How can the optimal number of clusters can be derived from inertia?</li>
</ul>
</td>
</tr>
</table>

<table>
<tr>
<td style="border-style: none"><img src="./img/0_reference.png" height="100px"></td>
<td style="border-style: none">&nbsp;&nbsp;</td>
<td style="border-style: none; vertical-align: middle"><u>Further reading:</u>
<ul>
<li>Chapter 20 (p 337ff) from "Data Science from Scratch"</li>
<li>Scikit Learn on <a href="https://scikit-learn.org/stable/modules/clustering.html">Clustering</a></li>
</ul>
</td>
</tr>
</table>

### Affinity Propagation

`Affinity propagation` does not require to define the numbers of clusters in advance.

It is based on node similarities and _message passing_ between adjacent, similar nodes.

<br>
<img src="./img/6_affinity_1.png" width="400px"><br><br>
<img src="./img/6_affinity_2.png" width="400px"><br><br>
<span style="font-size: 70%">Source: <a href="http://www.ijeee.net/uploadfile/2014/0807/20140807114023665.pdf">Map/Reduce Affinity Propagation Clustering Algorithm, Wei-Chih Hung, Chun-Yen Chu, and Yi-Leh Wu (2015)</a></span>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.cluster import AffinityPropagation

from sklearn.metrics import silhouette_score, \
                            homogeneity_score, \
                            completeness_score, \
                            v_measure_score, \
                            adjusted_rand_score

In [ ]:
# we use the data above (X, X_train, X_test)

# train model
af = AffinityPropagation(preference=-100, random_state=1).fit(X_train)
k = len(af.cluster_centers_)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_
print(f"Number of clusters: {k}")
print(f"Cluster centers:\n{af.cluster_centers_}")

In [ ]:
# visualize
import seaborn as sns
colors = sns.color_palette("tab10")

for c, col in zip(range(k), colors):
    x = X_train[:, 0][af.labels_ == c]
    y = X_train[:, 1][af.labels_ == c]
    plt.scatter(af.cluster_centers_[c][0],
                af.cluster_centers_[c][1],
                s=80, color=col, edgecolors="#000000")
    plt.scatter(x, y, alpha=0.5, color=col)
    for i in range(len(x)):
        plt.plot([x[i], af.cluster_centers_[c][0]], 
                 [y[i], af.cluster_centers_[c][1]], 
                 alpha=0.5, color=col, linewidth=1.0)

# plot test data
plt.scatter(X_test[:, 0], X_test[:, 1], s=20, alpha=0.5, color="#666666")
plt.title(f"Estimated number of clusters: {k}")
plt.show()


`Affinity` clustering can be used in recommender systems to cluster similar items (based on interest, preferences or other metrics)

<table>
<tr>
<td style="border-style: none"><img src="./img/0_reference.png" height="100px"></td>
<td style="border-style: none">&nbsp;&nbsp;</td>
<td style="border-style: none; vertical-align: middle"><u>Further reading:</u>
<ul>
<li>Scikit Learn on <a href="https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation">affinity propagation clustering</a></li>
</ul>
</td>
</tr>
</table>

### Hierarchical clustering

Hierarchical cluster algorithms group data in trees (`dendrograms`). Two basic approaches distinguish the cluster algorithm:

- __Agglomerative__ (bottom up): starts from the closest pair in the data, merging additional data to it

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <img src="./img/6_Agglomerative_Clustering.png" width="500px">

- __Divisive__ (top down): all data are member of the primary cluster that gets split recursively

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <img src="./img/6_Divisive_Clustering.png" width="500px">

<br><span style="font-size: 70%">Source: <a href="https://www.datacamp.com/tutorial/introduction-hierarchical-clustering-python">DataCamp: An Introduction to Hierarchical Clustering in Python</a></span>

##### Measuring distance

Hierarchical clusters measure the `distance` between pairs of cluster members.

__Single linkage__:

$ distance (C_i, C_j) = min \left\{ d (X_i, X_j) \right\} $ &nbsp; ... with $ i, j $ denoting Clusters

<br>

__Complete linkage__:

$ distance (C_i, C_j) = max \left\{ d (X_i, X_j) \right\} $ 

<br>

__Average linkage__:

$ distance (C_i, C_j) = \dfrac{ \sum \left\{ d (X_i, X_j) \right\} }{n} $  &nbsp; ... with $ n $ total number of distances

<br>

`Scikit learn` supports agglomerative clustering.

<table>
<tr>
<td style="border-style: none"><img src="./img/0_reference.png" height="100px"></td>
<td style="border-style: none">&nbsp;&nbsp;</td>
<td style="border-style: none; vertical-align: middle"><u>Further reading:</u>
<ul>
<li>DataCamp on <a href="https://www.datacamp.com/tutorial/introduction-hierarchical-clustering-python">hierarchical clustering</a></li>
<li>Scikit Learn on <a href="https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering">agglomerative clustering</a></li>
</ul>
</td>
</tr>
</table>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import kneighbors_graph

from sklearn.cluster import AgglomerativeClustering, FeatureAgglomeration

from sklearn.inspection import DecisionBoundaryDisplay


In [ ]:
# we use the data above (only X, as agglomerative clustering cannot predict unseen data - doesn't make sense)
k = 5

# train model
agg = AgglomerativeClustering(n_clusters=k).fit(X)
c_pred = agg.fit_predict(X)

In [ ]:
# visualize
import seaborn as sns
colors = sns.color_palette("tab10")

# DecisionBoundaryDisplay.from_estimator(agg, X, alpha=0.3, response_method="fit_predict")
for c, col in zip(range(k), colors):
    x = X[:, 0][c_pred == c]
    y = X[:, 1][c_pred == c]
    plt.scatter(x, y, alpha=0.5)
plt.title("Agglomeration cluster")
plt.show()

`Agglomerative clustering` assigns clusters a little different (see intersection of red / blue here and orange / blue in the affinity graph).

<br>

### Density-based spatial clustering of applications (DBSCAN)

`DBSCAN`, given a set of points, it groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away).

<img src="./img/6_DBSCAN.png" width="400px"><br><br>

<span style="font-size: 70%">In this diagram, minPts = 4. Point <b>A</b> and the other <b>red points</b> are <b>core points</b>, because the area surrounding these points in an &epsilon; radius contain at least 4 points (including the point itself). Because they are all reachable from one another, they form a single cluster.<br> Points <b>B</b> and <b>C</b> are not core points, but are <b>reachable from A</b> (via other core points) and thus belong to the cluster as well. <br>Point <b>N</b> is a <b>noise point</b> that is neither a core point nor directly-reachable.<br>Source: <a href="https://en.wikipedia.org/wiki/DBSCAN">Wikipedia</a></span>

<br>

__DBSCAN pseudo code__:

<pre>DBSCAN(DB, distFunc, eps, minPts) {
    C := 0                                                  /* Cluster counter */
    for each point P in database DB {
        if label(P) ≠ undefined then continue               /* Previously processed in inner loop */
        Neighbors N := RangeQuery(DB, distFunc, P, eps)     /* Find neighbors */
        if |N| < minPts then {                              /* Density check */
            label(P) := Noise                               /* Label as Noise */
            continue
        }
        C := C + 1                                          /* next cluster label */
        label(P) := C                                       /* Label initial point */
        SeedSet S := N \ {P}                                /* Neighbors to expand */
        for each point Q in S {                             /* Process every seed point Q */
            if label(Q) = Noise then label(Q) := C          /* Change Noise to border point */
            if label(Q) ≠ undefined then continue           /* Previously processed (e.g., border point) */
            label(Q) := C                                   /* Label neighbor */
            Neighbors N := RangeQuery(DB, distFunc, Q, eps) /* Find neighbors */
            if |N| ≥ minPts then {                          /* Density check (if Q is a core point) */
                S := S ∪ N                                  /* Add new neighbors to seed set */
            }
        }
    }
}

RangeQuery(DB, distFunc, Q, eps) {
    Neighbors N := empty list
    for each point P in database DB {                      /* Scan all points in the database */
        if distFunc(Q, P) ≤ eps then {                     /* Compute distance and check epsilon */
            N := N ∪ {P}                                   /* Add to result */
        }
    }
    return N
}</pre>

<br>

`DBSCAN` can find non-linearly separable clusters. 

<img src="./img/6_DBSCAN_2.png" width="250px"><br><br>

<span style="font-size: 70%">This dataset cannot be adequately clustered with k-means or Gaussian Mixture EM clustering.<br>Source: <a href="https://en.wikipedia.org/wiki/DBSCAN">Wikipedia</a></span>

<br>

Let's give it a try.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import DBSCAN

from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
# we use the data above

# train model
dbs = DBSCAN().fit(X)
labels = dbs.labels_
k = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print(f"Estimated number of clusters: {k}")
print(f"Estimated number of noise points: {n_noise_}")

In [ ]:
# visualize
import seaborn as sns
colors = sns.color_palette("tab10")

for c, col in zip(range(k), colors):
    x = X[:, 0][labels == c]
    y = X[:, 1][labels == c]
    plt.scatter(x, y, s=20)
plt.scatter(X[:, 0][labels == -1], X[:, 1][labels == -1], s=20, color="#666666", alpha=0.5)
plt.title(f"DBSCAN finds k={k} clusters and {n_noise_} noise points")
plt.show()


<span style="font-size: 128px">&#9749;</span> Coffee break!